# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy import stats
#importing the models
import Kmeans
import ALS
import NN
import Surprize
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV

# Final Functions

In [ ]:
def median_vote(predictions):
    #computing the median
    median=np.median(predictions,axis=0)
    #making sure the result is an int (not the case if the amount of predictions is even)
    bounded_median=np.floor(median)
    return bounded_median

In [ ]:
def mode_vote(predictions):
    #computing the mode
    return stats.mode(predictions,axis=0)[0]

In [ ]:
def maj_vote(predictions):
    top=predictions.iloc[0][:,np.newaxis]
    mode=stats.mode(predictions,axis=0)
    freq=mode[1].T/predictions.shape[0]
    return np.where(freq<0.5,top,mode[0].T)

In [ ]:
def cluster_vote(predictions):
    return np.round(np.mean(predictions,0))

In [ ]:
def vote(voting_f):
    #useful constants
    submission_path='submission.csv'
    training_path = "data/data_train.csv"
    format_path = "data/sampleSubmission.csv"
    #Loading the data
    print("Loading datasets")
    try:
        input_ = pd.read_csv(training_path)
        format_ = pd.read_csv(format_path)
    except FileNotFoundError:
        print("Impossible to load training or format files, "
              "please double check")
        return pd.DataFrame([])
    #computing the prediction of the ALS algorithm
    predictions=ALS.main(input_.copy(), format_.copy())
    print(predictions.head(),format_.head())
    #computing multiple predictions of the kmeans algorithm
    for k in [6]:
        predictions=predictions.merge(Kmeans.main(input_.copy(), format_.copy(), k),on='Id')
        print(predictions.shape)
    #computing the prediction of the NN algorithm
    predictions=predictions.merge(NN.main(input_.copy(), format_.copy()),on='Id')
    print(predictions.shape)
    #setting 'Id' as the index of the aggregation of predictions
    predictions.set_index('Id', inplace=True)
    #finding the best prediction through the voting function
    print('Voting...')
    predictions['Prediction']=voting_f(predictions.T)
    #exporting the final prediction using the submission path
    print('Exporting the final prediction...')
    predictions[['Prediction']].to_csv(submission_path)
    print('Done!')
    return predictions[['Prediction']]

In [ ]:
coco=vote(cluster_vote)

# Find predictors weights

In [ ]:
#useful constants
submission_path='submission.csv'
training_path = "data/data_train.csv"
format_path = "data/sampleSubmission.csv"

In [ ]:
#Loading the data
print("Loading datasets")
try:
    input_ = pd.read_csv(training_path)
    format_ = pd.read_csv(format_path)
except FileNotFoundError:
    print("Impossible to load training or format files, "
          "please double check")

In [ ]:
#Splitting the data
np.random.seed(1)
train, test =sklearn.model_selection.train_test_split(input_,test_size=0.1)

In [ ]:
predictions_surprize = Surprize.main(train.copy(), test.copy())

In [ ]:
#computing the predictions of the ALS algorithm
predictions_als=ALS.main(train.copy(), test.copy())

In [ ]:
#computing the best prediction of the kmeans algorithm
k=6
predictions_kmeans = Kmeans.main(train.copy(), test.copy(), k).set_index("Id")

In [ ]:
#computing the prediction of the NN algorithm
predictions_nn = NN.main(train.copy(), test.copy()).set_index("Id")

In [ ]:
true_ratings = test.copy()
true_ratings.set_index("Id", inplace=True)
true_ratings.columns = ["y"]
concat = pd.concat([predictions_als, 
                    predictions_kmeans, 
                    predictions_nn, 
                    predictions_surprize, 
                    true_ratings], axis=1, sort=False)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# preds_train, preds_test = train_test_split(concat)
X = concat.loc[:,~(concat.columns == "y")]
y = concat.loc[:, "y"]
rr = RidgeCV(alphas=np.linspace(1e-5, 3, 1000), store_cv_values=True).fit(X,y)
predictor_coefficients = dict(zip(X.columns, rr.coef_))

# Final training and predictions

In [ ]:
predictions_surprize = Surprize.main(input_.copy(), format_.copy(), cache_name="final")
predictions_als=ALS.main(input_.copy(), format_.copy(), cache_name="final")

#computing the best prediction of the kmeans algorithm
k=6
predictions_kmeans = Kmeans.main(input_.copy(), format_.copy(), k)
predictions_nn = NN.main(input_.copy(), format_.copy())
true_ratings = test.copy()
true_ratings.set_index("Id", inplace=True)
true_ratings.columns = ["y"]
concat = pd.concat([predictions_als, 
                    predictions_kmeans, 
                    predictions_nn, 
                    predictions_surprize, 
                    true_ratings], axis=1, sort=False)

In [ ]:
predictions_als.head()
predictions_kmeans.head()


In [ ]:
concat